**ePartogram-GPT using Zephyr 7B Alpha and Pytorch NLP**

------------------------------------------------------

**System Setup**

In [3]:
#install required packages
!pip install -q transformers peft accelerate bitsandbytes safetensors sentencepiece streamlit chromadb langchain langchain-community langchain-core langchain-huggingface sentence-transformers gradio pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 24.2 MB/s et

In [4]:
# fixing unicode error in google colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [14]:
# import dependencies
import os
import gradio as gr
from google.colab import drive
import csv

import chromadb
#from langchain.llms import HuggingFacePipeline

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain import HuggingFacePipeline
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

**Define and Train Model**

In [6]:
# specify model huggingface mode name
model_name = "anakin87/zephyr-7b-alpha-sharded"

###### other models:
# "HuggingFaceH4/zephyr-7b-beta"

# function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )
    return model

In [7]:
# function for initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

In [8]:
# load model
model = load_quantized_model(model_name)

# initialize tokenizer
tokenizer = initialize_tokenizer(model_name)

# specify stop token ids
stop_token_ids = [0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [9]:
# mount google drive and specify folder path
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/Colab Notebooks/My_PDFs_AIML'

# try:
#     from google.colab import drive
#     drive.mount('/content/gdrive/')
#     folder_path = "/content/gdrive/MyDrive/MyDrive/International_Maternal_Healthcare_Textbook.pdf"
# except:
#     folder_path = "./International_Maternal_Healthcare_Textbook.pdf"

!ls '/content/drive/MyDrive/Colab Notebooks/My_PDFs_AIML'

Mounted at /content/drive
'Clinical Rules Table_4.25.16.pdf'
'Copy of eZazi Tech User Manual for DOCTOR V 2.0 (2) (1).pdf'
'Copy of eZazi User Manual for Healthworkers- English (1) (1).pdf'
'Copy of eZazi validation.pdf'
'Copy of WHOnextgenerationpartograph.pdf'
'effectiveness of Epartogram 2019.pdf'
'Epartogram editorial.pdf'
'Use of an electronic Partograph_Litwin.pdf'
'WHO_Labour Care Guide_English.pdf'
"WHO LCG User's Manual.pdf"


In [10]:
# load pdf files
loader = PyPDFDirectoryLoader(folder_path)
documents = loader.load()
print(documents)

# split the documents in small chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100) #Change the chunk_size and chunk_overlap as needed
all_splits = text_splitter.split_documents(documents)

# specify embedding model (using huggingface sentence transformer)
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name = embedding_model_name, model_kwargs = model_kwargs) #TO DO

#embed document chunks
vectordb = Chroma.from_documents(documents = all_splits, embedding = embeddings, persist_directory="chroma_db") #TO DO

# specify the retriever
retriever = vectordb.as_retriever()

[Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/My_PDFs_AIML/Copy of WHOnextgenerationpartograph.pdf', 'page': 0}, page_content='WHO next-generation partograph: revolutionary\nsteps towards individualised labour care\nGJ Hofmeyr,a,b S Bernitz,c,d M Bonet,e M Bucagu,f B Dao,g S Downe,h\nH Galadanci,i CSE Homer,j V Hundley,k T Lavender,l B Levy,m D Lissauer,n P Lumbiganon,o\nFE McConville,f R Pattinson,p Z Qureshi,q JP Souza,r ME Stanton,s P ten Hoope-Bender,t\nV Vannevel,p JP Vogel,j OT Oladapoe\na Department of Obstetrics and Gynaecology, University of Botswana, Gaborone, Botswana b Effective Care Research Unit, University of the\nWitwatersrand and Walter Sisulu University, East London, South Africa c Department of Obstetrics and Gynecology, Østfold Hospital Trust,\nGr/C23 alum, Norway d Faculty of Health Sciences, Oslo Metropolitan University, Oslo, Norway e UNDP/UNFPA/UNICEF/WHO/World Bank\nSpecial Programme of Research, Development and Research Training in Huma

<ipython-input-10-861146f92fb9>:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = embedding_model_name, model_kwargs = model_kwargs) #TO DO


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
# build huggingface pipeline for using zephyr-7b-alpha
pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=2048,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

# specify the llm
llm = HuggingFacePipeline(pipeline=pipeline)

# build conversational retrieval chain with memory (rag) using langchain
def create_conversation(query: str, chat_history: list) -> tuple:
    try:

        memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=False
        )
        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=retriever,
            memory=memory
        )

        result = qa_chain({'question': query, 'chat_history': chat_history})
        chat_history.append((query, result['answer']))
        return '', chat_history


    except Exception as e:
        chat_history.append((query, e))
        return '', chat_history

<ipython-input-11-8617d193030b>:17: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipeline)


### **Run Chatbot UI**

In [11]:
# build gradio ui
with gr.Blocks() as demo:

    # TO DO: Initialize a chatbot component in Gradio with a label like "Chat with your data (Zephyr 7B Alpha)"
    chatbot = gr.Chatbot(label="Chat with your data (Zephyr 7B Alpha)") # TO DO: Define the Gradio chatbot component

    # TO DO: Create a textbox for user input
    msg = gr.Textbox(label="Enter your message") # TO DO: Define the Gradio textbox component for user input

    # TO DO: Add a button to clear the chat
    clear = gr.ClearButton([msg, chatbot]) # TO DO: Define a clear button to reset the input and chatbot

    # TO DO: Set up the interaction between user input and the chatbot using msg.submit()
    # The function `create_conversation` will be called when the user submits a message

    # msg.submit(# TO DO: call the `create_conversation` function, [msg, chatbot], [msg, chatbot])
    msg.submit(create_conversation, [msg, chatbot], [msg, chatbot])

demo.launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:225: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://85edee53cdc796acd3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


------------------------------------------------------
------------------------------------------------------

Pre-Prompt Engineering

To Do - Add in reference to a document in RAG

In [15]:
def save_chat_to_csv(chat_history: list, filename: str = 'chat_history.csv'):
    with open(filename, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for query, response in chat_history:
            writer.writerow([query, response])

In [16]:
def pre_prompt_engineering(user_input: str) -> str:
    # Example: Add context or instructions
    context = "You are an expert medical assistant."
    return f"{context} {user_input}"

def create_conversation(query: str, chat_history: list) -> tuple:
    try:
        memory = ConversationBufferMemory(
            memory_key='chat_history', return_messages=False
        )
        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm, retriever=retriever, memory=memory
        )

        result = qa_chain({'question': query, 'chat_history': chat_history})
        chat_history.append((query, result['answer']))

        # Save the updated chat history to CSV
        save_chat_to_csv([(query, result['answer'])])

        return '', chat_history
    except Exception as e:
        error_message = str(e)
        chat_history.append((query, error_message))

        # Save the error to CSV as well
        save_chat_to_csv([(query, error_message)])

        return '', chat_history

In [17]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with your data (Zephyr 7B Alpha)")
    msg = gr.Textbox(label="Enter your message")
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(create_conversation, [msg, chatbot], [msg, chatbot])

demo.launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:225: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://886c4a8aa0dbfcd992.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
